In [1]:
# Math & data
import numpy as np
import pandas as pd
import itertools

In [2]:
df_feature_importance = pd.read_csv('feature_importance_XGBoost_Test roc_auc=0.789662.csv', sep=";")
df_feature_importance

,feature,importance
0,CODE_GENDER,0.006239
1,FLAG_OWN_CAR,0.003012
2,FLAG_OWN_REALTY,0.001540
3,CNT_CHILDREN,0.001103
4,AMT_INCOME_TOTAL,0.001184
...,...,...
790,CC%NAME_CONTRACT_STATUS__nan#MAX,0.000000
791,CC%NAME_CONTRACT_STATUS__nan#MEAN,0.000000
792,CC%NAME_CONTRACT_STATUS__nan#SUM,0.000000
793,CC%NAME_CONTRACT_STATUS__nan#VAR,0.000000


In [3]:
def get_root_feat(row):
    sep1 = '%'
    sep2 = '__'
    sep3 = '#'
    root_feat = row
    if sep1 in root_feat:
        root_feat = root_feat.split(sep1)[1]
    if sep2 in root_feat:
        root_feat = root_feat.split(sep2)[0]
    if sep3 in root_feat:
        root_feat = root_feat.split(sep3)[0]
    # print(f"root_feat = {root_feat}")
    return root_feat

def get_context(row):
    sep1 = '%'
#     sep2 = '__'
#     sep3 = '#'
    context = row
    if sep1 in context:
        tab = context.split(sep1)
        context = tab[0]+sep1
#         context = tab[1]
    return context

def get_operation(row):
    sep = '#'
    operation = np.nan
    if sep in row:
        operation = row.split(sep)[1].lower()
    return operation

def get_context_and_root_feat(row):
#     sep1 = '%'
#     sep2 = '__'
    sep3 = '#'
    context_and_root_feat = row
#     if sep1 in context_and_root_feat:
#         context_and_root_feat = root_feat.split(sep1)[1]
#     if sep2 in context_and_root_feat:
#         context_and_root_feat = context_and_root_feat.split(sep2)[0]
    if sep3 in context_and_root_feat:
        context_and_root_feat = context_and_root_feat.split(sep3)[0]
    # print(f"context_and_root_feat = {context_and_root_feat}")
    return context_and_root_feat

# Pré-traitements avant opérations

# Pre-treatments before operations

In [4]:
df_feature_importance['Root feature'] = df_feature_importance['feature'].apply(get_root_feat)
# df_feature_importance['Context'] = df_feature_importance['feature'].apply(get_context)
df_feature_importance['Context+root feature'] = df_feature_importance['feature'].apply(get_context_and_root_feat)
df_feature_importance['Operation'] = df_feature_importance['feature'].apply(get_operation)
df_feature_importance

,feature,importance,Root feature,Context+root feature,Operation
0,CODE_GENDER,0.006239,CODE_GENDER,CODE_GENDER,NaN
1,FLAG_OWN_CAR,0.003012,FLAG_OWN_CAR,FLAG_OWN_CAR,NaN
2,FLAG_OWN_REALTY,0.001540,FLAG_OWN_REALTY,FLAG_OWN_REALTY,NaN
3,CNT_CHILDREN,0.001103,CNT_CHILDREN,CNT_CHILDREN,NaN
4,AMT_INCOME_TOTAL,0.001184,AMT_INCOME_TOTAL,AMT_INCOME_TOTAL,NaN
...,...,...,...,...,...
790,CC%NAME_CONTRACT_STATUS__nan#MAX,0.000000,NAME_CONTRACT_STATUS,CC%NAME_CONTRACT_STATUS__nan,max
791,CC%NAME_CONTRACT_STATUS__nan#MEAN,0.000000,NAME_CONTRACT_STATUS,CC%NAME_CONTRACT_STATUS__nan,mean
792,CC%NAME_CONTRACT_STATUS__nan#SUM,0.000000,NAME_CONTRACT_STATUS,CC%NAME_CONTRACT_STATUS__nan,sum
793,CC%NAME_CONTRACT_STATUS__nan#VAR,0.000000,NAME_CONTRACT_STATUS,CC%NAME_CONTRACT_STATUS__nan,var


# Regroupement des importances pour les groupes de variables

# Grouping of importances for groups of variables

**Remarque :** On sait que des variables portent plusieurs fois le même nom dans les différentes tables, mais il est logique de les traiter de façon homogène.

**Note:** We know that variables have the same name several times in the different tables, but it is logical to treat them in a homogeneous way.

In [5]:
# df_agg_ctx_root = df_feature_importance.groupby('Context+root feature')['importance'].agg(sum).reset_index()
df_agg_root = df_feature_importance.groupby('Root feature')['importance'].agg(sum).reset_index()
df_agg_root.columns = ['Root feature', 'Root feature importance']
df_agg_root = df_agg_root.sort_values(by='Root feature importance', ascending=False)
df_agg_root.head(20)

,Root feature,Root feature importance
135,MONTHS_BALANCE,0.031063
0,AMT_ANNUITY,0.024346
138,NAME_CONTRACT_STATUS,0.024211
161,ORGANIZATION_TYPE,0.023146
35,APP_CREDIT_PERC,0.022777
166,PRODUCT_COMBINATION,0.021740
142,NAME_GOODS_CATEGORY,0.018657
15,AMT_GOODS_PRICE,0.018405
63,DAYS_DECISION,0.018281
60,DAYS_CREDIT,0.018241


In [6]:
df_agg_root.loc[df_agg_root['Root feature']=='EXT_SOURCE_3']

,Root feature,Root feature importance
83,EXT_SOURCE_3,0.016821


In [8]:
df_agg_root.to_csv('root_feature_importance.csv', sep=";", index=False)

# Repérage de l'opération la plus importante

# Identification of the most important operation

In [9]:
df_agg_best_op = df_feature_importance.groupby(['Context+root feature'])[['Context+root feature', 'importance']].agg({'importance': max}).reset_index()
df_agg_best_op.columns = ['Context+root feature', 'importance max']
df_agg_best_op.tail(50)

,Context+root feature,importance max
507,PREV%PRODUCT_COMBINATION__POS other with interest,0.001655
508,PREV%PRODUCT_COMBINATION__POS others without i...,0.000000
509,PREV%PRODUCT_COMBINATION__nan,0.000000
510,PREV%RATE_DOWN_PAYMENT,0.001698
511,PREV%WEEKDAY_APPR_PROCESS_START__FRIDAY,0.001115
512,PREV%WEEKDAY_APPR_PROCESS_START__MONDAY,0.001195
513,PREV%WEEKDAY_APPR_PROCESS_START__SATURDAY,0.000961
514,PREV%WEEKDAY_APPR_PROCESS_START__SUNDAY,0.001262
515,PREV%WEEKDAY_APPR_PROCESS_START__THURSDAY,0.001030
516,PREV%WEEKDAY_APPR_PROCESS_START__TUESDAY,0.001092


In [10]:
print(df_agg_best_op.shape)

(557, 2)


In [11]:
df_feature_importance = df_feature_importance.merge(df_agg_best_op, how='left', on='Context+root feature')
df_feature_importance['Best operation'] = (df_feature_importance['importance'] == df_feature_importance['importance max'])
df_feature_importance.tail(50)

,feature,importance,Root feature,Context+root feature,Operation,importance max,Best operation
745,CC%SK_DPD#MAX,0.001371,SK_DPD,CC%SK_DPD,max,0.001371,True
746,CC%SK_DPD#MEAN,0.001073,SK_DPD,CC%SK_DPD,mean,0.001371,False
747,CC%SK_DPD#SUM,0.000000,SK_DPD,CC%SK_DPD,sum,0.001371,False
748,CC%SK_DPD#VAR,0.001349,SK_DPD,CC%SK_DPD,var,0.001371,False
749,CC%SK_DPD_DEF#MIN,0.000000,SK_DPD_DEF,CC%SK_DPD_DEF,min,0.001744,False
750,CC%SK_DPD_DEF#MAX,0.001348,SK_DPD_DEF,CC%SK_DPD_DEF,max,0.001744,False
751,CC%SK_DPD_DEF#MEAN,0.001744,SK_DPD_DEF,CC%SK_DPD_DEF,mean,0.001744,True
752,CC%SK_DPD_DEF#SUM,0.000000,SK_DPD_DEF,CC%SK_DPD_DEF,sum,0.001744,False
753,CC%SK_DPD_DEF#VAR,0.000000,SK_DPD_DEF,CC%SK_DPD_DEF,var,0.001744,False
754,CC%NAME_CONTRACT_STATUS__Active#MIN,0.001257,NAME_CONTRACT_STATUS,CC%NAME_CONTRACT_STATUS__Active,min,0.001801,False


In [12]:
df_feature_importance.to_csv('best_operation.csv', sep=";", index=False)